In [ ]:
def is_colision(self, x, y):
        for cone in self.car.forward_cones:
            ex = cone.x - x; ey = cone.y - y
            if ex**2 + ey**2 - cone.radius**2 <= 0:
                return True
        return False

    def sense_obstacle(self):
        data = []
        points = []
        x = self.car.get_position()[0] + self.pos[0]
        y = self.car.get_position()[1] + self.pos[1]
        for angle in np.linspace(self.angle_min + self.car.state.yaw, self.angle_max + self.car.state.yaw,self.angle_num, True):
            sense = False
            x1 = x + self.range_min*math.cos(angle)
            y1 = y + self.range_min*math.sin(angle)

            x2 = x + self.range_max*math.cos(angle)
            y2 = y + self.range_max*math.sin(angle)

            for i in range(self.range_num+1):
                u = i/self.range_num
                x3 = x2*u + x1*(1 - u)
                y3 = y2*u + y1*(1 - u)
                if self.is_colision(x3, y3):
                    dis = self.distance((x3, y3))
                    data.append({'position': (x3, y3), 'distance': dis, 'angle': angle})
                    points.append((x3, y3))
                    sense = True
                    break
            if not sense:
                data.append({'position': None, 'distance': self.range_max, 'angle': angle})
        self.sense_data = data
        self.points = points

In [ ]:
def sense_obstacle_fast_angles(self, track):

        data = []
        points = []

        for angle in np.linspace(self.angle_min, self.angle_max, self.angle_num, True):
            direction = np.array([math.cos(angle), math.sin(angle)])
            closest_point = None
            min_dist = self.range_max

            for cone in self.car.forward_cones_lidar:
                circle_center = np.array([cone.x, cone.y])
                r = cone.radius
                hit = self.ray_circle_intersection(self.pos, direction, circle_center, r)
                if hit is not None:
                    dist = np.linalg.norm(hit - self.pos)
                    if dist < min_dist:
                        min_dist = dist
                        closest_point = hit
                        color = cone.color

            if closest_point is not None:
                data.append({'position': tuple(closest_point), 'distance': min_dist, 'angle': angle, 'color': color})
                points.append(tuple(closest_point))
            else:
                data.append({'position': None, 'distance': self.range_max, 'angle': angle, 'color': None})

        self.sense_data = data
        self.points = points


In [ ]:
def find_visible_cones(self):

        visible_cones = []

        forward_vec = np.array([np.cos(self.sense_yaw), np.sin(self.sense_yaw)])

        # Step 2: Check distance to each cone
        for cone in self.car.track.cones:

            dx = cone.x - self.car.state.x
            dy = cone.y - self.car.state.y
            dist = math.hypot(dx, dy)

            direct_to_cone = np.array([dx,dy])/dist
            dot_product = np.clip(np.dot(forward_vec, direct_to_cone), -1.0, 1.0)
            angle_deg = np.degrees(np.arccos(dot_product))


            if dist <= self.range_max and angle_deg <= 85:
                visible_cones.append(cone)
                self.seen_cones.add(cone)

        self.visible_cones = visible_cones

    def distance(self, obs_pose):
        ex = obs_pose[0] - self.pos[0]
        ey = obs_pose[1] - self.pos[1]
        return math.hypot(ex, ey)

In [ ]:
def fast_euclidean_clustering(self, distance_threshold=0.5, min_cluster_size=1):
        points = self.points
        tree = cKDTree(points)
        visited = np.zeros(len(points), dtype=bool)
        clusters = []

        for i in range(len(points)):
            if visited[i]:
                continue

            queue = [i]
            cluster = []

            while queue:
                idx = queue.pop()
                if visited[idx]:
                    continue
                visited[idx] = True
                cluster.append(idx)
                neighbors = tree.query_ball_point(points[idx], distance_threshold)
                queue.extend([n for n in neighbors if not visited[n]])

            if len(cluster) >= min_cluster_size:
                clusters.append(cluster)

        return clusters

In [ ]:
def plot_detections(self, clusters, subplot_index=111):
        # Map positions to indices in self.sense_data
        pos_to_data_index = {
            tuple(d['position']): i
            for i, d in enumerate(self.sense_data)
            if d['position'] is not None
        }

        # Generate a color for each cluster
        colors = cm.rainbow(np.linspace(0, 1, len(clusters)))

        ax = plt.subplot(subplot_index, polar=True)

        for cluster, color in zip(clusters, colors):
            cluster_angles = []
            cluster_distances = []
            for idx in cluster:
                pos = self.points[idx]
                data_idx = pos_to_data_index.get(tuple(pos))
                if data_idx is not None:
                    d = self.sense_data[data_idx]
                    cluster_angles.append(d['angle'])
                    cluster_distances.append(d['distance'])

            ax.scatter(cluster_angles, cluster_distances, s=10, color=color,
                    label=f'Cluster {clusters.index(cluster)}')

        ax.set_title("Clustered LiDAR Scan (Polar)")
        ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))

In [ ]:
    def estimate_cone_center(self, cluster_indices, known_radius=0.1):
        cluster = np.array([self.points[i] for i in cluster_indices])
        n = len(cluster)

        if n == 0:
            return None

        elif n == 1:
            idx = cluster_indices[0]
            point = self.points[idx]
            sense_idx = self.find_matching_sense_idx(point)
            sense = self.sense_data[sense_idx]
            x, y = sense['position']
            angle = sense['angle']
            dx = math.cos(angle)
            dy = math.sin(angle)
            return (x - known_radius * dx, y - known_radius * dy)

        elif n == 2:
            p1, p2 = cluster
            midpoint = (p1 + p2) / 2
            vec = p2 - p1
            norm_vec = np.array([-vec[1], vec[0]])  # perpendicular
            norm_vec = norm_vec / np.linalg.norm(norm_vec)
            center1 = midpoint + norm_vec * known_radius
            center2 = midpoint - norm_vec * known_radius
            return tuple(center1 if np.linalg.norm(center1) < np.linalg.norm(center2) else center2)

        else:
            first = complex(*cluster[0])
            middle = complex(*cluster[len(cluster) // 2])
            last = complex(*cluster[-1])
            pos = self.circle_from_3_points(first, middle, last) # pos, r
            return (pos.real, pos.imag)

In [ ]:
def get_detected_cones(self):
        """
        Converts self.cone_pos and cluster color info into a list of Cone objects.
        """
        detected_cones = []

        # Build a mapping from point (x, y) → index in sense_data
        pos_to_index = {
            tuple(d['position']): i
            for i, d in enumerate(self.sense_data)
            if d['position'] is not None
        }

        if self.clusters and self.cone_pos:
            for cluster, pos in zip(self.clusters, self.cone_pos):
                if pos is None:
                    continue

                # Count dominant color
                color_counts = Counter()
                for idx in cluster:
                    point = tuple(self.points[idx])
                    sense_idx = pos_to_index.get(point)
                    if sense_idx is not None:
                        color = self.sense_data[sense_idx].get('color', 'black')
                        color_counts[color] += 1

                dominant_color = color_counts.most_common(1)[0][0] if color_counts else 'black'
                cone = Cone(pos[0], pos[1], dominant_color)
                detected_cones.append(cone)

        return detected_cones

In [ ]:
def plot_lidar(self):
        plt.figure(figsize=(12, 6))  # wider to fit two plots

        # --- Subplot 1: Cartesian LiDAR View ---
        ax1 = plt.subplot(1, 2, 1)
        for i in range(len(self.sense_data)):
            angle = self.sense_angle_min + self.resolution * i
            x = self.pos[0]
            y = self.pos[1]
            rayx = [x, x + math.cos(angle) * self.sense_data[i]['distance']]
            rayy = [y, y + math.sin(angle) * self.sense_data[i]['distance']]
            ax1.plot(rayx, rayy, '-b', linewidth=0.5)


        # Build a mapping from point (x, y) → index in sense_data
        pos_to_index = {
            tuple(d['position']): i
            for i, d in enumerate(self.sense_data)
            if d['position'] is not None
        }

        # Plot cone centers with dominant cluster color
        if self.clusters and self.cone_pos:
            for cluster, pos in zip(self.clusters, self.cone_pos):
                if pos is None:
                    continue

                # Use original sense_data indices to get colors
                color_counts = Counter()
                for idx in cluster:
                    point = tuple(self.points[idx])
                    sense_idx = pos_to_index.get(point)
                    if sense_idx is not None:
                        color = self.sense_data[sense_idx].get('color', 'black')
                        color_counts[color] += 1

                dominant_color = color_counts.most_common(1)[0][0] if color_counts else 'black'
                circle = plt.Circle(pos, 0.1, color=dominant_color, fill=True)
                ax1.add_patch(circle)

        if self.car:
            ax1.scatter(self.car.state.x, self.car.state.y, c='purple', marker='s', label="Car")

        if self.car:
            ax1.scatter(self.pos[0], self.pos[1], c='cyan', marker='s', label="Lidar")

        ax1.set_title("LiDAR Vision (Cartesian)")
        ax1.set_aspect('equal')
        ax1.legend()

        # --- Subplot 2: Polar Cluster View ---
        self.plot_detections(self.clusters, subplot_index=122)

        plt.tight_layout()
        plt.show()

In [ ]:
def sense_obstacle_fast_cones(self, track):

        data = []
        visible_cones = []

        angles = np.linspace(self.sense_angle_min, self.sense_angle_max, self.angle_num, True)
        

        for angle in angles:
            data.append({
                'position': None,
                'distance': self.range_max,
                'angle': angle,
                'color': None
            })

        for cone in track.cones:

            cone = 
            
            vec_to_cone = np.array([cone.x - self.pos[0], cone.y - self.pos[1]])
            dist = np.linalg.norm(vec_to_cone)

            if dist > self.range_max:
                continue
            
            angle_to_cone = np.arctan2(vec_to_cone[1], vec_to_cone[0])
            relative_angle = angle_to_cone - self.sense_yaw
            relative_angle = np.arctan2(np.sin(relative_angle), np.cos(relative_angle))

            if (self.angle_max < relative_angle) or (relative_angle < self.angle_min):
                continue
            
            #visible_cones.append(cone)
            #self.seen_cones.add(cone)

            angular_radius = np.arcsin(cone.radius / dist)
            lower_bound = relative_angle - angular_radius
            upper_bound = relative_angle + angular_radius

            candidate_indices = np.where((angles-self.sense_yaw >= lower_bound) & (angles-self.sense_yaw <= upper_bound))[0]

            for idx in candidate_indices:
                ray_angle = angles[idx]
                ray_dir = np.array([np.cos(ray_angle), np.sin(ray_angle)])

                hit = self.ray_circle_intersection(self.pos, ray_dir, np.array([cone.x, cone.y]), cone.radius)
                
                if hit is not None:
                    hit_dist = np.linalg.norm(hit - self.pos)
                    if hit_dist < data[idx]['distance']:
                        # Update the stored data immediately
                        data[idx]['distance'] = hit_dist
                        data[idx]['position'] = tuple(hit)
                        data[idx]['color'] = cone.color

        self.sense_data = data

        # Convert sensed points to car frame
        self.relative_sense_data = []
        for d in data:
            if d['position'] is None:
                self.relative_sense_data.append({
                    'position': None,
                    'distance': self.range_max,
                    'angle': d['angle']-self.sense_yaw,
                    'color': None
                })
                continue
            px, py = d['position']
            dx = px - self.pos[0]
            dy = py - self.pos[1]

            # Rotate by -yaw (from world to car frame)
            cos_yaw = np.cos(-self.sense_yaw)
            sin_yaw = np.sin(-self.sense_yaw)
            rel_x = cos_yaw * dx - sin_yaw * dy
            rel_y = sin_yaw * dx + cos_yaw * dy

            self.relative_sense_data.append({
                'position': (rel_x, rel_y),
                'distance': d['distance'],
                'angle': d['angle']-self.sense_yaw,
                'color': d['color']
            })

        self.points = {
        'positions': [d['position'] for i, d in enumerate(self.relative_sense_data) if d['position'] is not None],
        'indices': [i for i, d in enumerate(self.relative_sense_data) if d['position'] is not None]
        }

In [ ]:
# def match_cones(self, max_dist=0.3):
    #     raw_matches = []
    #     distances = []

    #     for i, c_curr in enumerate(self.lidar.relative_cones):
    #         best_match = None
    #         min_dist = float('inf')
    #         for j, c_prev in enumerate(self.prev_lidar_data):
    #             if c_curr.color != c_prev.color:
    #                 continue
    #             dist = np.hypot(c_curr.x - c_prev.x, c_curr.y - c_prev.y)
    #             if dist < min_dist and dist < max_dist:
    #                 best_match = j
    #                 min_dist = dist
    #         if best_match is not None:
    #             raw_matches.append((best_match, i))  # (prev_idx, curr_idx)
    #             distances.append(min_dist)

    #     # Filter using 2 standard deviations above the mean
    #     if distances:
    #         mean = np.mean(distances)
    #         std_dev = np.std(distances)
    #         threshold = mean + 2 * std_dev
    #         filtered_matches = [
    #             match for match, dist in zip(raw_matches, distances) if dist <= threshold
    #         ]
    #     else:
    #         filtered_matches = []

    #     return filtered_matches


    # def match_lidar_points(self, max_dist=0.3):
    #     raw_matches = []
    #     distances = []

    #     for i, curr in enumerate(self.lidar.relative_sense_data):  # current frame
    #         best_match = None
    #         min_dist = float('inf')
    #         curr_x, curr_y = curr['position']

    #         for j, prev in enumerate(self.prev_lidar_data):  # previous frame
    #             prev_x, prev_y = prev['position']
    #             dist = np.hypot(curr_x - prev_x, curr_y - prev_y)
    #             if dist < min_dist and dist < max_dist:
    #                 best_match = j
    #                 min_dist = dist

    #         if best_match is not None:
    #             raw_matches.append((best_match, i))  # (prev_idx, curr_idx)
    #             distances.append(min_dist)

    #     # Filter using mean + 2 std dev
    #     if distances:
    #         mean = np.mean(distances)
    #         std_dev = np.std(distances)
    #         threshold = mean + 2 * std_dev
    #         filtered_matches = [
    #             match for match, dist in zip(raw_matches, distances) if dist <= threshold
    #         ]
    #     else:
    #         filtered_matches = []

    #     return filtered_matches

    

    # def estimate_transform(self, matches):
    #     if len(matches) < 2:
    #         return None

    #     prev_pts = np.array([[self.prev_lidar_data[i].x, self.prev_lidar_data[i].y] for i, _ in matches])
    #     curr_pts = np.array([[self.lidar.relative_cones[j].x, self.lidar.relative_cones[j].y] for _, j in matches])

    #     centroid_prev = np.mean(prev_pts, axis=0)
    #     centroid_curr = np.mean(curr_pts, axis=0)

    #     P = prev_pts - centroid_prev
    #     C = curr_pts - centroid_curr

    #     H = C.T @ P
    #     U, _, Vt = np.linalg.svd(H)
    #     R = Vt.T @ U.T

    #     if np.linalg.det(R) < 0:
    #         Vt[1, :] *= -1
    #         R = Vt.T @ U.T

    #     t = centroid_prev - R @ centroid_curr
    #     yaw = np.arctan2(R[1, 0], R[0, 0])

    #     return t[0], t[1], yaw

    # def estimate_transform(self, matches):
    #     if len(matches) < 2:
    #         return None

    #     # Extract matched points
    #     prev_pts = np.array([
    #         self.prev_lidar_data[i]['position'] for i, _ in matches
    #     ])
    #     curr_pts = np.array([
    #         self.lidar.relative_sense_data[j]['position'] for _, j in matches
    #     ])

    #     centroid_prev = np.mean(prev_pts, axis=0)
    #     centroid_curr = np.mean(curr_pts, axis=0)

    #     P = prev_pts - centroid_prev
    #     C = curr_pts - centroid_curr

    #     H = C.T @ P
    #     U, _, Vt = np.linalg.svd(H)
    #     R = Vt.T @ U.T

    #     if np.linalg.det(R) < 0:
    #         Vt[1, :] *= -1
    #         R = Vt.T @ U.T

    #     t = centroid_prev - R @ centroid_curr
    #     yaw = np.arctan2(R[1, 0], R[0, 0])

    #     return t[0], t[1], yaw



    # def update_pose(self, dx, dy, dyaw):
    #     R_global = np.array([
    #         [np.cos(self.state_e.yaw), -np.sin(self.state_e.yaw)],
    #         [np.sin(self.state_e.yaw),  np.cos(self.state_e.yaw)]
    #     ])
    #     delta_global = R_global @ np.array([dx, dy])
    #     x_new = self.state_e.x + delta_global[0]
    #     y_new = self.state_e.y + delta_global[1]
    #     yaw_new = self.state_e.yaw + dyaw

    #     self.state_e.x = x_new
    #     self.state_e.y = y_new
    #     self.state_e.yaw = yaw_new


    # def extract_transform(self, transformation):
    #     """
    #     Extract the translation (dx, dy) and yaw (dyaw) from the 3x3 transformation matrix.
    #     """
    #     # Extract rotation and translation
    #     R = transformation[:2, :2]
    #     t = transformation[:2, 2]

    #     # Compute yaw from the rotation matrix
    #     yaw = np.arctan2(R[1, 0], R[0, 0])

    #     # dx, dy are the translation components
    #     dx, dy = t[0], t[1]

    #     return dx, dy, yaw



    
    # def estimate_transform_icp(self, prev_pts, curr_pts, max_iterations=50, threshold=1e-6):
    #     """
    #     Perform 2D ICP (Iterative Closest Point) to estimate the transform between two point clouds.
    #     """
    #     # Start with identity transformation
    #     transformation = np.eye(3)
        
    #     for iteration in range(max_iterations):
    #         # Find closest points in current to previous
    #         closest_points = self.find_closest_points(prev_pts, curr_pts)
            
    #         # Estimate the rigid transformation (rotation and translation)
    #         R, t = self.compute_rigid_transform(prev_pts, closest_points)
            
    #         # Apply the transformation to the current points
    #         curr_pts = self.apply_transformation_to_points(curr_pts, R, t)
            
    #         # Compute the new transformation matrix
    #         transformation_new = np.eye(3)
    #         transformation_new[:2, :2] = R
    #         transformation_new[:2, 2] = t
            
    #         # Check for convergence
    #         if np.linalg.norm(transformation_new - transformation) < threshold:
    #             break
                
    #         transformation = transformation_new

    #     return transformation  # 3x3 transformation matrix (rotation + translation)

    # def find_closest_points(self, prev_pts, curr_pts):
    #     """
    #     Find the closest points in prev_pts for each point in curr_pts.
    #     """
    #     closest_points = []
    #     for curr_point in curr_pts:
    #         distances = np.linalg.norm(prev_pts - curr_point, axis=1)
    #         closest_idx = np.argmin(distances)
    #         closest_points.append(prev_pts[closest_idx])
    #     return np.array(closest_points)
    
    # def compute_rigid_transform(self, A, B):
    #     """
    #     Compute the rigid transform (rotation and translation) that aligns A to B in 2D.
    #     """
    #     centroid_A = np.mean(A, axis=0)
    #     centroid_B = np.mean(B, axis=0)
        
    #     A_centered = A - centroid_A
    #     B_centered = B - centroid_B
        
    #     H = A_centered.T @ B_centered
    #     U, _, Vt = np.linalg.svd(H)
    #     R = Vt.T @ U.T
        
    #     if np.linalg.det(R) < 0:
    #         Vt[1, :] *= -1
    #         R = Vt.T @ U.T
        
    #     t = centroid_B - R @ centroid_A
        
    #     return R, t


    # def apply_transformation_to_points(self, points, R, t):
    #     """
    #     Apply the transformation (rotation and translation) to a set of points.
    #     """
    #     return np.dot(points, R.T) + t


    # def apply_transformation(self, point, transform):
    #     """
    #     Apply the transformation to a single point.
    #     """
    #     point_homogeneous = np.append(point, 1)  # Convert to homogeneous coordinates
    #     transformed_point = transform @ point_homogeneous
    #     return transformed_point[:2]  # Return only x, y




    # def match_lidar_points(self, max_dist=0.3, max_iterations=50, threshold=1e-6):
    #     """
    #     Match lidar points using ICP method and return matched points.
    #     """
    #     raw_matches = []
    #     distances = []

    #     # Get the previous and current lidar data
    #     prev_points = np.array([cone['position'] for cone in self.prev_lidar_data])  # previous frame
    #     curr_points = np.array([cone['position'] for cone in self.lidar.relative_sense_data])  # current frame
        
    #     # Perform ICP to estimate the transformation
    #     transform = self.estimate_transform_icp(prev_points, curr_points, max_iterations, threshold)

    #     if transform is not None:
    #         # The transformation (dx, dy, dyaw) is embedded in this transformation
    #         dx, dy, dyaw = self.extract_transform(transform)

    #         # Now apply the transformation to the pose
    #         self.update_pose(dx, dy, dyaw)

    #         return raw_matches  # Matching is now done, and the pose is updated.

    #     return []  # Return empty if transformation is not valid

In [ ]:
# def point_based_matching(self, point_pairs):
    #     """
    #     This function is based on the paper "Robot Pose Estimation in Unknown Environments by Matching 2D Range Scans"
    #     by F. Lu and E. Milios.

    #     :param point_pairs: the matched point pairs [((x1, y1), (x1', y1')), ..., ((xi, yi), (xi', yi')), ...]
    #     :return: the rotation angle and the 2D translation (x, y) to be applied for matching the given pairs of points
    #     """

    #     x_mean = 0
    #     y_mean = 0
    #     xp_mean = 0
    #     yp_mean = 0
    #     n = len(point_pairs)

    #     if n == 0:
    #         return None, None, None

    #     for pair in point_pairs:

    #         (x, y), (xp, yp) = pair

    #         x_mean += x
    #         y_mean += y
    #         xp_mean += xp
    #         yp_mean += yp

    #     x_mean /= n
    #     y_mean /= n
    #     xp_mean /= n
    #     yp_mean /= n

    #     s_x_xp = 0
    #     s_y_yp = 0
    #     s_x_yp = 0
    #     s_y_xp = 0
    #     for pair in point_pairs:

    #         (x, y), (xp, yp) = pair

    #         s_x_xp += (x - x_mean)*(xp - xp_mean)
    #         s_y_yp += (y - y_mean)*(yp - yp_mean)
    #         s_x_yp += (x - x_mean)*(yp - yp_mean)
    #         s_y_xp += (y - y_mean)*(xp - xp_mean)

    #     rot_angle = math.atan2(s_x_yp - s_y_xp, s_x_xp + s_y_yp)
    #     translation_x = xp_mean - (x_mean*math.cos(rot_angle) - y_mean*math.sin(rot_angle))
    #     translation_y = yp_mean - (x_mean*math.sin(rot_angle) + y_mean*math.cos(rot_angle))

    #     return rot_angle, translation_x, translation_y

In [ ]:
#  ICP parameters
EPS = 0.0001
MAX_ITER = 100

show_animation = False


def icp_matching(previous_points, current_points):
    """
    Iterative Closest Point matching
    - input
    previous_points: 2D or 3D points in the previous frame
    current_points: 2D or 3D points in the current frame
    - output
    R: Rotation matrix
    T: Translation vector
    """
    H = None  # homogeneous transformation matrix

    dError = np.inf
    preError = np.inf
    count = 0

    if show_animation:
        fig = plt.figure()
        if previous_points.shape[0] == 3:
           fig.add_subplot(111, projection='3d')

    while dError >= EPS:
        count += 1

        if show_animation:  # pragma: no cover
            plot_points(previous_points, current_points, fig)
            plt.pause(0.1)

        indexes, error = nearest_neighbor_association(previous_points, current_points)
        Rt, Tt = svd_motion_estimation(previous_points[:, indexes], current_points)
        # update current points
        current_points = (Rt @ current_points) + Tt[:, np.newaxis]

        dError = preError - error
        print("Residual:", error)

        if dError < 0:  # prevent matrix H changing, exit loop
            print("Not Converge...", preError, dError, count)
            break

        preError = error
        H = update_homogeneous_matrix(H, Rt, Tt)

        if dError <= EPS:
            print("Converge", error, dError, count)
            break
        elif MAX_ITER <= count:
            print("Not Converge...", error, dError, count)
            break

    R = np.array(H[0:-1, 0:-1])
    T = np.array(H[0:-1, -1])

    return R, T

def icp_matching_f(previous_points, current_points):
    """
    Iterative Closest Point matching
    - input
    previous_points: 2D or 3D points in the previous frame
    current_points: 2D or 3D points in the current frame
    - output
    R: Rotation matrix
    T: Translation vector
    """
    H = None  # homogeneous transformation matrix

    dError = np.inf
    preError = np.inf
    count = 0

        # Check if the number of points differs
    n_points_prev = previous_points.shape[1]
    n_points_curr = current_points.shape[1]

    # Match points based on the current set's size
    if n_points_prev > n_points_curr:
        previous_points = previous_points[:, :n_points_curr]  # Down-sample the larger point cloud
    elif n_points_curr > n_points_prev:
        current_points = current_points[:, :n_points_prev]  # Down-sample the larger point cloud


    while dError >= EPS:
        count += 1

        indexes, error = nearest_neighbor_association(previous_points, current_points)
        # Slice based on indexes to align points
        previous_matched_points = previous_points[:, indexes]
        current_matched_points = current_points

        Rt, Tt = svd_motion_estimation_f(previous_matched_points, current_matched_points)

        # Apply the transformation to the current points
        current_points = (Rt @ current_points) + Tt[:, np.newaxis]

        dError = preError - error
        if dError < 0:
            print("Not Converged...", preError, dError, count)
            break

        preError = error
        H = update_homogeneous_matrix(H, Rt, Tt)

        if dError <= EPS:
            print("Converged", error, dError, count)
            break
        elif MAX_ITER <= count:
            print("Not Converged...", error, dError, count)
            break

    R = np.array(H[0:-1, 0:-1])
    T = np.array(H[0:-1, -1])

    return R, T



def svd_motion_estimation_f(previous_points, current_points):
    """
    Estimate the rigid body transformation (rotation and translation) between previous_points and current_points.
    Returns:
    - R: rotation matrix
    - t: translation vector
    """

    
    pm = np.mean(previous_points, axis=1)
    cm = np.mean(current_points, axis=1)

    p_shift = previous_points - pm[:, np.newaxis]
    c_shift = current_points - cm[:, np.newaxis]

    # Compute the covariance matrix
    W = c_shift @ p_shift.T
    u, s, vh = np.linalg.svd(W)

    R = (u @ vh).T
    t = pm - (R @ cm)

    return R, t


def downsample_to_match_size(reference_points, target_points):
    """
    Downsample the larger point cloud so that both have the same number of points.
    
    Inputs:
    - reference_points: np.ndarray of shape (D, N1)
    - target_points: np.ndarray of shape (D, N2)
    
    Returns:
    - reference_points_resized: shape (D, N)
    - target_points_resized: shape (D, N)
    """
    N1 = reference_points.shape[1]
    N2 = target_points.shape[1]
    N = min(N1, N2)

    if N1 > N:
        indices = np.random.choice(N1, N, replace=False)
        reference_points = reference_points[:, indices]
    if N2 > N:
        indices = np.random.choice(N2, N, replace=False)
        target_points = target_points[:, indices]

    return reference_points, target_points




def update_homogeneous_matrix(Hin, R, T):

    r_size = R.shape[0]
    H = np.zeros((r_size + 1, r_size + 1))

    H[0:r_size, 0:r_size] = R
    H[0:r_size, r_size] = T
    H[r_size, r_size] = 1.0

    if Hin is None:
        return H
    else:
        return Hin @ H

def nearest_neighbor_association_f(previous_points, current_points):
    """
    Associate each point in current_points to its nearest neighbor in previous_points.
    Handles differing numbers of points.
    Returns:
    - indexes: indices of closest previous_points
    - error: sum of distances (residual error)
    """

    # KD-Tree for fast NN lookup
    tree = cKDTree(previous_points.T)
    distances, indexes = tree.query(current_points.T)  # Query the nearest neighbors

    # Total ICP residual error
    error = np.sum(distances)

    return indexes, error


def nearest_neighbor_association(previous_points, current_points):

    # calc the sum of residual errors
    delta_points = previous_points - current_points
    d = np.linalg.norm(delta_points, axis=0)
    error = sum(d)

    # calc index with nearest neighbor assosiation
    d = np.linalg.norm(np.repeat(current_points, previous_points.shape[1], axis=1)
                       - np.tile(previous_points, (1, current_points.shape[1])), axis=0)
    indexes = np.argmin(d.reshape(current_points.shape[1], previous_points.shape[1]), axis=1)

    return indexes, error


def svd_motion_estimation(previous_points, current_points):
    pm = np.mean(previous_points, axis=1)
    cm = np.mean(current_points, axis=1)

    p_shift = previous_points - pm[:, np.newaxis]
    c_shift = current_points - cm[:, np.newaxis]

    W = c_shift @ p_shift.T
    u, s, vh = np.linalg.svd(W)

    R = (u @ vh).T
    t = pm - (R @ cm)

    return R, t


def plot_points(previous_points, current_points, figure):
    # for stopping simulation with the esc key.
    plt.gcf().canvas.mpl_connect(
        'key_release_event',
        lambda event: [exit(0) if event.key == 'escape' else None])
    if previous_points.shape[0] == 3:
        plt.clf()
        axes = figure.add_subplot(111, projection='3d')
        axes.scatter(previous_points[0, :], previous_points[1, :],
                    previous_points[2, :], c="r", marker=".")
        axes.scatter(current_points[0, :], current_points[1, :],
                    current_points[2, :], c="b", marker=".")
        axes.scatter(0.0, 0.0, 0.0, c="r", marker="x")
        figure.canvas.draw()
    else:
        plt.cla()
        plt.plot(previous_points[0, :], previous_points[1, :], ".r")
        plt.plot(current_points[0, :], current_points[1, :], ".b")
        plt.plot(0.0, 0.0, "xr")
        plt.axis("equal")

In [ ]:
                #stacked_points = np.vstack((np.array([p['position'] for p in self.prev_lidar_data])[:,0], np.array([p['position'] for p in self.prev_lidar_data])[:,1])) 
                #stacked_points_c = np.vstack((np.array([p['position'] for p in self.lidar.relative_sense_data])[:,0], np.array([p['position'] for p in self.lidar.relative_sense_data])[:,1])) 
                #R, T = icp_matching_f(stacked_points,stacked_points_c)
                #self.apply_transformation_to_pose_d(R,T)
                #self.plot_matched_cones_by_index(matches)

In [ ]:
# Particle and FastSLAM scaffolding based on your LiDAR and car model
import numpy as np
from scipy.stats import multivariate_normal

class Particle:
    def __init__(self, pose, weight=1.0):
        self.pose = np.array(pose)  # [x, y, theta]
        self.weight = weight
        self.landmarks = {}  # {landmark_id: (mean: np.array([x, y]), covariance: np.array([[2x2]]))}

    def predict(self, control_input, dt, motion_noise):
        """
        Apply noisy motion model to update pose
        control_input: [delta_x, delta_y, delta_theta]
        """
        noise = np.random.normal(0, motion_noise, 3)
        dx, dy, dtheta = control_input + noise
        x, y, theta = self.pose
        self.pose = np.array([x + dx, y + dy, theta + dtheta])

    def update_landmark(self, landmark_id, z_rel, R):
        """
        EKF update of landmark (in world frame) given observation z_rel in car frame
        """
        # Transform relative measurement to global coords
        lx = self.pose[0] + np.cos(self.pose[2]) * z_rel[0] - np.sin(self.pose[2]) * z_rel[1]
        ly = self.pose[1] + np.sin(self.pose[2]) * z_rel[0] + np.cos(self.pose[2]) * z_rel[1]
        z = np.array([lx, ly])

        if landmark_id not in self.landmarks:
            self.landmarks[landmark_id] = (z, R)
            return

        mu, sigma = self.landmarks[landmark_id]

        # Measurement prediction is just mu
        H = np.eye(2)
        S = H @ sigma @ H.T + R
        K = sigma @ H.T @ np.linalg.inv(S)
        mu_new = mu + K @ (z - mu)
        sigma_new = (np.eye(2) - K @ H) @ sigma
        self.landmarks[landmark_id] = (mu_new, sigma_new)

        # Likelihood (used for weight update)
        likelihood = multivariate_normal.pdf(z, mean=mu, cov=S)
        self.weight *= likelihood


def initialize_particles(n, init_pose, noise_std):
    return [Particle(pose=init_pose + np.random.normal(0, noise_std, 3)) for _ in range(n)]

def resample_particles(particles):
    weights = np.array([p.weight for p in particles])
    weights /= np.sum(weights)
    indices = np.random.choice(len(particles), size=len(particles), p=weights)
    resampled = [particles[i] for i in indices]
    for p in resampled:
        p.weight = 1.0
    return resampled

In [ ]:
# def compute_mahalanobis_distance(particle, z, lm_id, Q_cov):
#     xf = particle.lm[lm_id].reshape(2, 1)
#     Pf = particle.lmP[2 * lm_id:2 * lm_id + 2]
#     zp, Hv, Hf, Sf = compute_jacobians(particle, xf, Pf, Q_cov)
#     dz = z[0:2].reshape(2, 1) - zp
#     dz[1, 0] = pi_2_pi(dz[1, 0])
#     d2 = float(dz.T @ np.linalg.inv(Sf) @ dz)
#     return d2

# def update_with_observation(particles, z, mahal_thresh=9.21):
#     for iz in range(z.shape[1]):
#         for p in particles:
#             best_lm_id = None
#             best_d2 = float('inf')

#             for lm_id in range(p.lm.shape[0]):
#                 if np.allclose(p.lm[lm_id], 0):
#                     continue

#                 # ✴️ New: Euclidean range filter
#                 dx = p.lm[lm_id, 0] - p.state.x
#                 dy = p.lm[lm_id, 1] - p.state.y
#                 dist = math.hypot(dx, dy)
#                 if dist > MAX_RANGE + 1.0:  # optional buffer (1.0 m)
#                     continue
                
#                 d2 = compute_mahalanobis_distance(p, z[:, iz], lm_id, Q)
#                 if d2 < mahal_thresh and d2 < best_d2:
#                     best_d2 = d2
#                     best_lm_id = lm_id

#             if best_lm_id is not None:
#                 z_with_id = np.array([z[0, iz], z[1, iz], best_lm_id])
#                 w = compute_weight(p, z_with_id, Q)
#                 p.w *= w
#                 p = update_landmark(p, z_with_id, Q)
#                 p = proposal_sampling(p, z_with_id, Q)
#             else:
#                 new_id = np.where(np.all(p.lm == 0, axis=1))[0]
#                 if len(new_id) == 0:
#                     continue
#                 z_with_id = np.array([z[0, iz], z[1, iz], new_id[0]])
#                 p = add_new_lm(p, z_with_id, Q)
#     return particles

In [ ]:
# def update_with_observation(particles, z):
#     for iz in range(len(z[0, :])):
#         landmark_id = int(z[2, iz])

#         for ip in range(N_PARTICLE):
#             # new landmark
#             if abs(particles[ip].lm[landmark_id, 0]) <= 0.01:
#                 particles[ip] = add_new_lm(particles[ip], z[:, iz], Q)
#             # known landmark
#             else:
#                 w = compute_weight(particles[ip], z[:, iz], Q)
#                 particles[ip].w *= w

#                 particles[ip] = update_landmark(particles[ip], z[:, iz], Q)
#                 particles[ip] = proposal_sampling(particles[ip], z[:, iz], Q)

#     return particles

In [ ]:
# def proposal_sampling(particle, z, Q_cov):
#     lm_id = int(z[2])
#     xf = particle.lm[lm_id, :].reshape(2, 1)
#     Pf = particle.lmP[2 * lm_id:2 * lm_id + 2]
#     # State
#     x = np.array([particle.state.x, particle.state.y, particle.state.yaw]).reshape(3, 1)
#     P = particle.P
#     zp, Hv, Hf, Sf = compute_jacobians(particle, xf, Pf, Q_cov)

#     Sfi = np.linalg.inv(Sf)
#     dz = z[0:2].reshape(2, 1) - zp
#     dz[1] = pi_2_pi(dz[1])

#     Pi = np.linalg.inv(P + R)

#     particle.P = np.linalg.inv(Hv.T @ Sfi @ Hv + Pi)  # proposal covariance
#     x += particle.P @ Hv.T @ Sfi @ dz  # proposal mean

#     particle.state.x = x[0, 0]
#     particle.state.y = x[1, 0]
#     particle.state.yaw = x[2, 0]

#     return particle

In [ ]:
# def update_landmark(particle, z, Q_cov):
#     lm_id = int(z[2])
#     xf = np.array(particle.lm[lm_id]).reshape(2, 1)
#     Pf = particle.lmP[lm_id]

#     zp, Hv, Hf, Sf = compute_jacobians(particle, xf, Pf, Q_cov)

#     dz = z[0:2].reshape(2, 1) - zp
#     dz[1, 0] = pi_2_pi(dz[1, 0])

#     xf, Pf = update_kf_with_cholesky(xf, Pf, dz, Q, Hf)

#     particle.lm[lm_id, :] = xf.T
#     particle.lmP[2 * lm_id:2 * lm_id + 2, :] = Pf

#     return particle

In [ ]:
# def add_new_lm(particle, z, Q_cov):
#     r = z[0]
#     b = z[1]
#     lm_id = int(z[2])

#     s = math.sin(pi_2_pi(particle.state.yaw + b))
#     c = math.cos(pi_2_pi(particle.state.yaw + b))

#     particle.lm[lm_id, 0] = particle.state.x + r * c
#     particle.lm[lm_id, 1] = particle.state.y + r * s

#     # covariance
#     dx = r * c
#     dy = r * s
#     d2 = dx ** 2 + dy ** 2
#     d = math.sqrt(d2)
#     Gz = np.array([[dx / d, dy / d],
#                    [-dy / d2, dx / d2]])
#     particle.lmP[2 * lm_id:2 * lm_id + 2] = np.linalg.inv(
#         Gz) @ Q_cov @ np.linalg.inv(Gz.T)

#     return particle

In [ ]:
def euclidean_distance(point1, point2):
        """
        Euclidean distance between two points.
        :param point1: the first point as a tuple (a_1, a_2, ..., a_n)
        :param point2: the second point as a tuple (b_1, b_2, ..., b_n)
        :return: the Euclidean distance
        """
        a = np.array(point1)
        b = np.array(point2)

        return np.linalg.norm(a - b, ord=2)


    def point_based_matching(self, point_pairs):
        """
        Computes the optimal 2D rigid transformation (rotation + translation) using SVD.
        :param point_pairs: list of point pairs [((x, y), (xp, yp)), ...]
        :return: (rotation_angle, translation_x, translation_y)
        """ 
        if len(point_pairs) == 0:
            return None, None, None

        # Convert to numpy arrays
        src = np.array([p[0] for p in point_pairs])  # original points
        dst = np.array([p[1] for p in point_pairs])  # transformed (target) points

        # Compute centroids
        src_mean = np.mean(src, axis=0)
        dst_mean = np.mean(dst, axis=0)

        # Center the points
        src_centered = src - src_mean
        dst_centered = dst - dst_mean

        # Compute covariance matrix
        H = src_centered.T @ dst_centered

        # Compute SVD
        U, _, Vt = np.linalg.svd(H)

        # Compute rotation matrix
        R = Vt.T @ U.T

        # Handle reflection case
        if np.linalg.det(R) < 0:
            Vt[1, :] *= -1
            R = Vt.T @ U.T

        # Extract angle from rotation matrix
        rot_angle = np.arctan2(R[1, 0], R[0, 0])

        # Compute translation
        t = dst_mean - R @ src_mean

        return rot_angle, t[0], t[1]



    def icp(self, reference_points, points, max_iterations=200, distance_threshold=0.3, convergence_translation_threshold=1e-19,
            convergence_rotation_threshold=1e-19, point_pairs_threshold=10, verbose=False):
        """
        An implementation of the Iterative Closest Point algorithm that matches a set of M 2D points to another set
        of N 2D (reference) points.

        :param reference_points: the reference point set as a numpy array (N x 2)
        :param points: the point that should be aligned to the reference_points set as a numpy array (M x 2)
        :param max_iterations: the maximum number of iteration to be executed
        :param distance_threshold: the distance threshold between two points in order to be considered as a pair
        :param convergence_translation_threshold: the threshold for the translation parameters (x and y) for the
                                                transformation to be considered converged
        :param convergence_rotation_threshold: the threshold for the rotation angle (in rad) for the transformation
                                                to be considered converged
        :param point_pairs_threshold: the minimum number of point pairs the should exist
        :param verbose: whether to print informative messages about the process (default: False)
        :return: the transformation history as a list of numpy arrays containing the rotation (R) and translation (T)
                transformation in each iteration in the format [R | T] and the aligned points as a numpy array M x 2
        """

        transformation_history = []

        nbrs = NearestNeighbors(n_neighbors=1, algorithm='kd_tree').fit(reference_points)

        for iter_num in range(max_iterations):
            if verbose:
                print('------ iteration', iter_num, '------')

            closest_point_pairs = []  # list of point correspondences for closest point rule

            distances, indices = nbrs.kneighbors(points)
            for nn_index in range(len(distances)):
                if distances[nn_index][0] < distance_threshold:
                    closest_point_pairs.append((points[nn_index], reference_points[indices[nn_index][0]]))

            # if only few point pairs, stop process
            if verbose:
                print('number of pairs found:', len(closest_point_pairs))
            if len(closest_point_pairs) < point_pairs_threshold:
                if verbose:
                    print('No better solution can be found (very few point pairs)!')
                break

            # compute translation and rotation using point correspondences
            closest_rot_angle, closest_translation_x, closest_translation_y = self.point_based_matching(closest_point_pairs)
            if closest_rot_angle is not None:
                if verbose:
                    print('Rotation:', math.degrees(closest_rot_angle), 'degrees')
                    print('Translation:', closest_translation_x, closest_translation_y)
            if closest_rot_angle is None or closest_translation_x is None or closest_translation_y is None:
                if verbose:
                    print('No better solution can be found!')
                break

            # transform 'points' (using the calculated rotation and translation)
            c, s = math.cos(closest_rot_angle), math.sin(closest_rot_angle)
            rot = np.array([[c, -s],
                            [s, c]])
            aligned_points = np.dot(points, rot.T)
            aligned_points[:, 0] += closest_translation_x
            aligned_points[:, 1] += closest_translation_y

            # update 'points' for the next iteration
            points = aligned_points

            # update transformation history
            transformation_history.append(np.hstack((rot, np.array([[closest_translation_x], [closest_translation_y]]))))

            # check convergence
            if (abs(closest_rot_angle) < convergence_rotation_threshold) \
                    and (abs(closest_translation_x) < convergence_translation_threshold) \
                    and (abs(closest_translation_y) < convergence_translation_threshold):
                if verbose:
                    print('Converged!')
                break

        return transformation_history, points


    def match_lidar_points(self, max_iterations=50, threshold=1, verbose=False):
        """
        Match lidar points using the ICP method and return matched points.
        """
        raw_matches = []

        # Get the previous and current lidar data
        prev_points = np.array([cone['position'] for cone in self.prev_lidar_filt])  # previous frame
        curr_points = np.array([cone['position'] for cone in self.lidar.sense_filt])  # current frame


        # Perform ICP to estimate the transformation
        transformation_history, aligned_points = self.icp(prev_points, curr_points, max_iterations, threshold, verbose=verbose)

        self.plot_transformation(prev_points, curr_points, aligned_points)

        if len(transformation_history) > 0:
            # Start with identity transformation
            overall_transform = np.eye(3)

            for transform in transformation_history:
                # Convert 2x3 [R | t] into a full 3x3 homogeneous transform
                T = np.eye(3)
                T[:2, :2] = transform[:, :2]
                T[:2, 2] = transform[:, 2]

                # Compose transformations
                overall_transform = overall_transform @ T

            # Apply the overall transformation to the pose
            self.apply_transformation_to_pose(overall_transform)

            return raw_matches

        return []
    
    def apply_transformation_to_pose(self, transformation):
        """
        Apply transformation from ICP (which is in the car's frame) to update pose in the global/inertial frame.
        """
        R = transformation[:2, :2]
        t = transformation[:2, 2]

        # Step 1: Convert the translation from body frame to inertial frame
        theta = self.state_e.yaw  # current orientation of the car
        rotation_to_global = np.array([
            [np.cos(theta), -np.sin(theta)],
            [np.sin(theta),  np.cos(theta)]
        ])
        t_global = rotation_to_global @ t

        # Step 2: Apply translation in inertial frame
        self.state_e.x += t_global[0]
        self.state_e.y += t_global[1]

        # Step 3: Update yaw with the rotation from ICP
        delta_yaw = np.arctan2(R[1, 0], R[0, 0])
        self.state_e.yaw += delta_yaw


    def apply_transformation_to_pose_d(self, R, T):
        """
        Apply transformation from ICP (which is in the car's frame) to update pose in the global/inertial frame.
        """

        # Step 1: Convert the translation from body frame to inertial frame
        theta = self.state_d.yaw  # current orientation of the car
        rotation_to_global = np.array([
            [np.cos(theta), -np.sin(theta)],
            [np.sin(theta),  np.cos(theta)]
        ])
        t_global = rotation_to_global @ T

        # Step 2: Apply translation in inertial frame
        self.state_d.x += t_global[0]
        self.state_d.y += t_global[1]

        # Step 3: Update yaw with the rotation from ICP
        delta_yaw = np.arctan2(R[1, 0], R[0, 0])
        self.state_d.yaw += delta_yaw

In [ ]:
def plot_matched_cones_by_index(self, matches):
        plt.figure(figsize=(8, 6))
        
        for prev_idx, curr_idx in matches:
            prev_point = self.prev_lidar_data[prev_idx]['position']
            curr_point = self.lidar.relative_sense_data[curr_idx]['position']

            if prev_point is None or curr_point is None:
                continue
                
            # Plot previous point
            plt.scatter(prev_point[0], prev_point[1], color='blue', label='Previous' if 'Previous' not in plt.gca().get_legend_handles_labels()[1] else "")
            
            # Plot current point
            plt.scatter(curr_point[0], curr_point[1], color='red', label='Current' if 'Current' not in plt.gca().get_legend_handles_labels()[1] else "")

            # Draw line between matched points
            plt.plot([prev_point[0], curr_point[0]], [prev_point[1], curr_point[1]], 'k--', linewidth=1)

        plt.xlabel('X [m]')
        plt.ylabel('Y [m]')
        plt.title('Matched Points Across Frames (Index-Based)')
        plt.legend()
        plt.axis('equal')
        plt.grid(True)
        plt.show()

In [ ]:
# def update_tracked_cones(self, current_cones, tracked_cones, inactive_cones, next_cone_id, frame_num, dist_thresh=0.5, max_missed=10):
    #     assigned_ids = []
    #     unmatched = set(range(len(current_cones)))

    #     # Step 1: Attempt to match current cones to tracked cones
    #     for t_cone in tracked_cones:
    #         best_j = -1
    #         best_dist = float('inf')
    #         for j in unmatched:
    #             c = current_cones[j]
    #             dist = np.linalg.norm([t_cone.x - c.x, t_cone.y - c.y])
    #             if dist < best_dist and dist < dist_thresh and c.color == t_cone.color:
    #                 best_dist = dist
    #                 best_j = j
    #         if best_j != -1:
    #             c = current_cones[best_j]
    #             # Update matched tracked cone
    #             t_cone.x = c.x
    #             t_cone.y = c.y
    #             t_cone.last_seen = frame_num
    #             t_cone.missed_count = 0  # Reset missed count
    #             assigned_ids.append((c, t_cone.id))
    #             unmatched.remove(best_j)
    #         else:
    #             # No match found → increment missed count
    #             t_cone.missed_count += 1

    #     # Step 2: Retire cones that have been missed too long
    #     active_cones = []
    #     for cone in tracked_cones:
    #         if cone.missed_count <= max_missed:
    #             active_cones.append(cone)
    #         else:
    #             inactive_cones.append(cone)
    #     tracked_cones[:] = active_cones

    #     # Step 3: Add unmatched cones as new tracked cones
    #     for j in unmatched:
    #         c = current_cones[j]
    #         new_cone = TrackedCone(c.x, c.y, c.color, next_cone_id)
    #         new_cone.last_seen = frame_num
    #         new_cone.missed_count = 0
    #         tracked_cones.append(new_cone)
    #         assigned_ids.append((c, next_cone_id))
    #         next_cone_id += 1

    #     return assigned_ids, tracked_cones, inactive_cones, next_cone_id

In [ ]:
def update_tracked_cones(self, current_cones, tracked_cones, inactive_cones, next_cone_id, frame_num,
                         dist_thresh=0.5, max_missed=10, dx=0.0, dy=0.0, dyaw=0.0):
        assigned_ids = []
        unmatched = set(range(len(current_cones)))

        # Step 0: Predict tracked_cone positions into current frame
        cos_yaw = np.cos(-dyaw)
        sin_yaw = np.sin(-dyaw)
        for t_cone in tracked_cones:
            # Apply inverse motion (cone moves relative to car)
            x_shifted = t_cone.x - dx
            y_shifted = t_cone.y - dy
            t_cone.x = cos_yaw * x_shifted - sin_yaw * y_shifted
            t_cone.y = sin_yaw * x_shifted + cos_yaw * y_shifted

        # Step 1: Attempt to match current cones to predicted tracked cones
        for t_cone in tracked_cones:
            best_j = -1
            best_dist = float('inf')
            for j in unmatched:
                c = current_cones[j]
                dist = np.linalg.norm([t_cone.x - c.x, t_cone.y - c.y])
                if dist < best_dist and dist < dist_thresh and c.color == t_cone.color:
                    best_dist = dist
                    best_j = j
            if best_j != -1:
                c = current_cones[best_j]
                # Update matched tracked cone
                t_cone.x = c.x
                t_cone.y = c.y
                t_cone.last_seen = frame_num
                t_cone.missed_count = 0
                assigned_ids.append((c, t_cone.id))
                unmatched.remove(best_j)
            else:
                t_cone.missed_count += 1

        # Step 2: Retire missed cones
        active_cones = [cone for cone in tracked_cones if cone.missed_count <= max_missed]
        inactive_cones.extend([cone for cone in tracked_cones if cone.missed_count > max_missed])
        tracked_cones[:] = active_cones

        # Step 3: Add unmatched cones as new tracked cones
        for j in unmatched:
            c = current_cones[j]
            new_cone = TrackedCone(c.x, c.y, c.color, next_cone_id)
            new_cone.last_seen = frame_num
            tracked_cones.append(new_cone)
            assigned_ids.append((c, next_cone_id))
            next_cone_id += 1

        return assigned_ids, tracked_cones, inactive_cones, next_cone_id

In [ ]:
    def initialise_tracked_cones(self, initial_cones, frame_num=0):

        for c in initial_cones:
            tracked = TrackedCone(c.x, c.y, c.color, self.next_cone_id)
            tracked.last_seen = frame_num
            tracked.missed_count = 0
            self.tracked_cones.append(tracked)
            self.next_cone_id += 1

    self.initialise_tracked_cones(self.lidar_data)

In [ ]:
# def plotting(self):

    #         plt.figure()

    #         yellow_positions_a = np.array([cone.get_position() for cone in self.map if cone.color == 'yellow'])
    #         blue_positions_a = np.array([cone.get_position() for cone in self.map if cone.color == 'blue'])


    #         if len(self.car.track.start_line) > 0:
    #             start_line = np.array([cone.get_position() for cone in self.car.track.start_line])
    #             plt.scatter(start_line[:, 0], start_line[:, 1], c='orange', marker='o', label="Start Cones")


    #         if len(yellow_positions_a) > 0:
    #             plt.scatter(yellow_positions_a[:, 0], yellow_positions_a[:, 1], c='yellow', marker='^', label="Seen Yellow Cones")  # Plot yellow and blue cones seen
    #         if len(blue_positions_a) > 0:
    #             plt.scatter(blue_positions_a[:, 0], blue_positions_a[:, 1], c='blue', marker='^', label="Seen Blue Cones")

    #         if len(self.path_taken) > 1:
    #             xs, ys = zip(*self.path_taken)
    #             plt.plot(xs, ys, label="Car Path", color="purple", linewidth=2)             # Plot the trajectory as a line

    #         if hasattr(self, "particles"):
    #             particle_xs = [p.state.x for p in self.particles]
    #             particle_ys = [p.state.y for p in self.particles]
    #             particle_ws = [p.w for p in self.particles]

    #             # Normalize weights to [0, 1]
    #             max_w = max(particle_ws)
    #             min_w = min(particle_ws)
    #             if max_w > min_w:
    #                 normalized_ws = [(w - min_w) / (max_w - min_w) for w in particle_ws]
    #             else:
    #                 normalized_ws = [0.0 for _ in particle_ws]  # all weights are equal

    #             # Use red for high weights, green for low (reverse colormap)
    #             colors = plt.cm.RdYlGn_r(normalized_ws)

    #             plt.scatter(particle_xs, particle_ys, c=colors, s=10, alpha=0.7, label="Particles")         

            
    #         if hasattr(self, "particles") and len(self.particles) > 0:
    #             n_landmarks = self.particles[0].lm.shape[0]
    #             mean_lms = []

    #             for lm_id in range(n_landmarks):
    #                 lm_positions = []
    #                 for p in self.particles:
    #                     if np.any(p.lm[lm_id] != 0):  # Only include if landmark was observed
    #                         lm_positions.append(p.lm[lm_id])
    #                 if lm_positions:
    #                     mean_pos = np.mean(lm_positions, axis=0)
    #                     mean_lms.append(mean_pos)

    #             if mean_lms:
    #                 mean_lms = np.array(mean_lms)
    #                 plt.scatter(mean_lms[:, 0], mean_lms[:, 1], c='black', marker='x', label='Mean Landmark Position')

    #         target = self.car_to_world(self.controller.target)
    #         plt.scatter(target[0], target[1], c='red', marker='x', label="Target")
    #         plt.scatter(self.car.state.x, self.car.state.y, c='purple', marker='s', label="Car")                                            # Plot car

    #         #if self.controller:
    #         #    self.controller.plot_control()


    #         plt.axis('equal')
    #         plt.legend()
    #         plt.title("Car Simulation")
    #         plt.show()

    #         if self.camera:
    #             self.camera.plot_camera()

    #         if self.lidar:
    #             self.lidar.plot_lidar()

                        #     # === FastSLAM Step ===
                    # control_input = np.array([0.0, 0.0, 0.0])  # Replace with car motion delta if you have odometry
                    # dt = 0.02
                    # motion_noise = 0.05

                    # for p in self.particles:
                    #     p.predict(control_input, dt, motion_noise)
                    #     for cone in self.lidar.cones:
                    #         relative_pos = self.world_to_car((cone.x, cone.y))
                    #         p.update_landmark(cone.color, np.array(relative_pos), R=np.eye(2)*0.05)

                    # self.particles = resample_particles(self.particles)
                    # self.estimated_pose = np.mean([p.pose for p in self.particles], axis=0)